In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import requests
import os
from io import BytesIO
from io import StringIO
from datetime import date, datetime, timezone, timedelta
!pip install datawrapper
from datawrapper import Datawrapper
DW_SECRET = os.getenv('dw_secret')
dw = Datawrapper(access_token=DW_SECRET)

In [ ]:
# Funktion zum Erstellen der URL basierend auf dem aktuellen Quartal im aktuellen Jahr
def generate_url():
    today = datetime.today()
    current_month = today.month
    year = today.year

    # Bestimme den Startmonat des aktuellen Quartals
    if current_month in [1, 2, 3]:
        quarter_start_month = 1
    elif current_month in [4, 5, 6]:
        quarter_start_month = 4
    elif current_month in [7, 8, 9]:
        quarter_start_month = 7
    else:
        quarter_start_month = 10

    # Erzeuge die URL
    url = f"https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ27/fz27_{year}{quarter_start_month:02d}.xlsx?__blob=publicationFile"

    # Prüfe, ob die Datei existiert
    try:
        response = requests.head(url, timeout=5)
        if response.status_code == 200:
            return url
    except requests.RequestException:
        pass

    # Wenn keine gültige URL im aktuellen Jahr vorhanden ist, gib None zurück
    return None

# URL zur Excel-Datei ermitteln
url_mob_best = generate_url()

if url_mob_best:
    # Excel-Datei herunterladen
    response = requests.get(url_mob_best)
    excel_data = BytesIO(response.content)

    # Daten aus dem Excel-Sheet einlesen
    df_2 = pd.read_excel(excel_data, sheet_name="FZ 27.9", engine='openpyxl', header=11).dropna(axis=1, how='all')

    # Relevanten Datenbereich auswählen (bis zur ersten komplett leeren Zeile)
    empty_row_index = df_2.index[df_2.isnull().all(axis=1)].min()
    df_2 = df_2.iloc[:empty_row_index]

    # Spalten umbenennen
    neue_spalten_liste = ['Quartal', 'Anzahl insgesamt', 'AA - Anzahl insgesamt', 'AA - Anteil in %',
                         'EA - Anzahl insgesamt', 'EA - Anteil in %', 'Elektro (BEV)',
                         'Brennstoffzelle (Wasserstoff)', 'Plug-in-Hybrid', 'HoP - Anzahl insgesamt',
                         'Benzin-Hybrid', 'Diesel-Hybrid', 'Gas insgesamt', 'Wasserstoff']
    df_2.rename(columns=dict(zip(df_2.columns, neue_spalten_liste)), inplace=True)


None


In [ ]:
def german_month_to_digit(month_name):
    german_months = {
        "januar": "01", "jan": "01",
        "februar": "02", "feb": "02",
        "märz": "03", "maerz": "03", "mrz": "03",
        "april": "04", "apr": "04",
        "mai": "05",
        "juni": "06", "jun": "06",
        "juli": "07", "jul": "07",
        "august": "08", "aug": "08",
        "september": "09", "sep": "09", "sept": "09",
        "oktober": "10", "okt": "10",
        "november": "11", "nov": "11",
        "dezember": "12", "dez": "12"
    }

    key = month_name.strip().lower()
    return german_months.get(key)


In [ ]:
## KARTE E-PKW

# URL zur JSON-Datei ohne Geometrie
#url = "https://stg-arcgisazurecdataprodeu1.az.arcgis.com/exportfiles-5853-88/FZ%20Pkw%20mit%20Elektroantrieb%20Gemeinde_-6454229268231740875.csv?sv=2018-03-28&sr=b&sig=p42kuYhDna7Fl78ScSASltpmP0DtQdc2YDIhPS3nGio%3D&se=2025-06-24T13%3A02%3A49Z&sp=r"
url = "https://services-eu1.arcgis.com/U09msXRZoxesNntH/arcgis/rest/services/FZ%20Pkw%20mit%20Elektroantrieb%20Gemeinde/FeatureServer/replicafilescache/FZ%20Pkw%20mit%20Elektroantrieb%20Gemeinde_-6454229268231740875.csv"
# JSON-Daten abrufen
response = requests.get(url)
csv_data = BytesIO(response.content)

# Einlesen des festgelegten Bereichs
df = pd.read_csv(csv_data, delimiter=',', decimal='.')

# Spalte in string konvertieren, falls nötig
df['Berichtszeitpunkt'] = df['Berichtszeitpunkt'].astype(str)

# Spalte in Datum umwandeln, z. B. '2023.1' → '2023-01'
df['Berichtszeitpunkt_parsed'] = pd.to_datetime(df['Berichtszeitpunkt'], format='%Y.%m', errors='coerce')

# Ungültige entfernen
df = df.dropna(subset=['Berichtszeitpunkt_parsed'])

# Aktuellstes Datum finden
aktueller_wert = df['Berichtszeitpunkt_parsed'].max()

# Zeilen mit diesem Datum auswählen
df = df[df['Berichtszeitpunkt_parsed'] == aktueller_wert]

# Optional: Datum wieder als 'YYYY.M' darstellen
aktueller_wert_str = aktueller_wert.strftime('%m.%Y')  # '%-m' für Monatszahl ohne führende Null (nur auf Unix/Mac); Windows braucht '%#m'

df = df[['AGS', 'Gemeinde', 'Pkw Elektro Anteil', 'Bezeichnung']]

# URL zur Excel-Datei "Deutschlandatlas"
url = "https://www.deutschlandatlas.bund.de/SharedDocs/Downloads/DE/Deutschlandatlas-Daten.xlsx?__blob=publicationFile&"

# Excel-Datei per HTTP anfordern
response = requests.get(url)
excel_data = BytesIO(response.content)

# Gewünschte Blätter und Spalten laden
blatt1 = pd.read_excel(excel_data, sheet_name="Deutschlandatlas_GEM1222", usecols=["GKZ1222", "bev_dicht"])
blatt2 = pd.read_excel(excel_data, sheet_name="Deutschlandatlas_GEM1221", usecols=["GKZ1221", "p_nelade", "p_selade"])

# Optional: Umbenennen der GKZ-Spalten, damit sie zusammengeführt werden können
blatt1 = blatt1.rename(columns={"GKZ1222": "AGS"})
blatt2 = blatt2.rename(columns={"GKZ1221": "AGS"})

# Merge der beiden Tabellen über die gemeinsame GKZ-Spalte
df_d = pd.merge(blatt1, blatt2, on="AGS", how="inner")
df_d = df_d.rename(columns={"bev_dicht": "Bevölkerungsdichte (2022)", "p_nelade": "Fahrzeit Normalladesäule (2023)", "p_selade": "Fahrzeit Schnellladesäule (2023)"})

df = pd.merge(df, df_d, on="AGS", how="left")

notes_text = f"* Unter Elektrofahrzeugen werden hier batterieelektrische Fahrzeuge (BEV), Brennstoffzelle (FCEV) und Plug-in Hybrid (PHEV) zusammengefasst. <br>Stand: {aktueller_wert_str}"

# Chart aktualisieren
CHART_ID = "NmAop"
dw.update_chart(CHART_ID, metadata={
    'annotate': {
        'notes': notes_text
    }
})
dw.publish_chart(chart_id=CHART_ID, display=False)


In [ ]:
##Neuzulassungen
def create_link(year, month):
    formatted_month = str(month).zfill(2)
    sheet_name = "FZ 28.2"
    return f"https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ28/fz28_{year}_{formatted_month}.xlsx?__blob=publicationFile&", sheet_name

def read_excel_with_check(content, sheet_name):
    # Bytes in einen Datei-ähnlichen Stream umwandeln
    excel_file = BytesIO(content)
    df = pd.read_excel(excel_file, sheet_name=sheet_name, skiprows=11, engine='openpyxl')
    if "Jahr 2024 insgesamt" in df.columns:
        excel_file.seek(0)  # Stream zurücksetzen
        df = pd.read_excel(excel_file, sheet_name=sheet_name, skiprows=10, engine='openpyxl')
    return df

# -------------------------------
# Neueste verfügbare Monatsdatei suchen
# -------------------------------
now = datetime.now()
year, month = now.year, now.month
df_1 = None

while year >= 2024 and df_1 is None:
    while month >= 1:
        link, sheet_name = create_link(year, month)
        response = requests.get(link)

        # Check if the response content type is a valid Excel file
        if response.status_code == 200 and 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' in response.headers.get('Content-Type', ''):
            try:
                temp = read_excel_with_check(response.content, sheet_name)

                # Only keep rows with "Jahr ... insgesamt"
                temp = temp[temp['Unnamed: 1'].astype(str).str.contains(r"^Jahr \d{4} insgesamt", na=False)]

                # Only keep relevant columns
                keep_cols = ['Unnamed: 1', 'Benzin-Hybrid', 'Diesel-Hybrid']
                available_cols = [col for col in keep_cols if col in temp.columns]
                df_1 = temp[available_cols].copy()

                # "Jahr XXXX insgesamt" → only year number
                df_1['Unnamed: 1'] = df_1['Unnamed: 1'].str.extract(r"(\d{4})")

                # Rename column
                df_1.rename(columns={'Unnamed: 1': 'Jahr'}, inplace=True)
                break
            except Exception as e:
                print(f"Error processing Excel file from {link}: {e}")
        else:
            print(f"Skipping {link}: Not a valid Excel file or no data available.")

        month -= 1
    year -= 1
    month = 12

## Zweiter Datensatz

def lade_excel(sheet="FZ 14.2.1"):
    jahr = datetime.now().year
    url_template = "https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ14/fz14_{jahr}.xlsx?__blob=publicationFile"

    while jahr >= 2000:
        r = requests.get(url_template.format(jahr=jahr))
        if r.status_code == 200 and 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' in r.headers.get('Content-Type', ''):
            try:
                excel_file = BytesIO(r.content)
                df = pd.read_excel(excel_file, sheet_name=sheet, header=[7, 8], engine="openpyxl")

                # Clean up column names
                df.columns = [
                    ' '.join(str(c).strip().replace('\n', '') for c in col if pd.notna(c))
                    if all("Unnamed:" not in str(c) for c in col) else str(col[0]).replace('\n', '')
                    for col in df.columns
                ]

                df = df.iloc[:, 1:]  # remove first column
                return df, jahr
            except Exception as e:
                print(f"Error processing Excel file from {url_template.format(jahr=jahr)}: {e}")

        jahr -= 1

    return None, None

# Usage
df, jahr = lade_excel()

if df is not None:
    df.columns = df.columns.str.strip()  # Removes leading and trailing spaces from all column names

    if "Jahr" in df.columns:
        df["Jahr"] = pd.to_numeric(df["Jahr"], errors="coerce")
        df = df[df["Jahr"] <= jahr].copy() # Selection of the correct section

    # Only keep years from 2016 onwards
    df = df[df['Jahr'] >= 2016].copy()

    # Operations
    # 1. New column "Gas (LPG, CNG)" = Sum of both columns
    df["Gas (LPG, CNG)"] = df["Flüssiggas (LPG)(einschl. bivalent)"].fillna(0) + df["Erdgas (CNG)(einschl. bivalent)"].fillna(0)

    # 2. New column "Hybrid o. Plug-in" = "Hybrid insgesamt" - "Hybrid dar. Plug-in"
    df["Hybrid (ohne Plug-in)"] = (
        pd.to_numeric(df["Hybrid insgesamt"], errors='coerce').fillna(0)
        - pd.to_numeric(df["Hybrid dar. Plug-in"], errors='coerce').fillna(0)
    )


    # 3. Rename columns
    df.rename(columns={"Hybrid dar. Plug-in": "Plug-in Hybrid "}, inplace=True)
    df.rename(columns={"Elektro(BEV)": "Elektro (BEV)"}, inplace=True)

    # Merge df with df_1 on the column 'Jahr'
    if df_1 is not None:
        # Convert 'Jahr' column in df_1 to numeric
        df_1['Jahr'] = pd.to_numeric(df_1['Jahr'], errors='coerce')
        # Merge df mit df_1 auf 'Jahr', um die Hybrid-Werte zu bekommen
        merged = df.merge(df_1[['Jahr', 'Diesel-Hybrid', 'Benzin-Hybrid']], on='Jahr', how='left')

        # Subtrahiere die Hybrid-Werte
        merged['Benzin'] = merged['Benzin'] + merged['Benzin-Hybrid']
        merged['Diesel'] = merged['Diesel'] + merged['Diesel-Hybrid']

        # Berechne die Summe der Hybrid-Werte
        merged['Hybrid_sum'] = merged['Diesel-Hybrid'] + merged['Benzin-Hybrid']

        # Berechne die Differenz zwischen df und df_1
        diff = merged['Hybrid (ohne Plug-in)'] - merged['Hybrid_sum']

        # Differenz zu bestehender 'Sonstige'-Spalte addieren
        merged['Sonstige'] = merged['Sonstige'] + diff

        # Optional: keine negativen Werte erlauben
        merged['Sonstige'] = merged['Sonstige'].apply(lambda x: x if x > 0 else 0)

        # Hilfsspalten entfernen
        merged.drop(columns=['Diesel-Hybrid', 'Benzin-Hybrid', 'Hybrid_sum'], inplace=True)

        # Ergebnis zurück in df speichern
        df = merged
    spalten_zu_behalten = ["Jahr", "Benzin", "Diesel", "Elektro (BEV)", "Gas (LPG, CNG)", "Plug-in Hybrid ", "Sonstige"]
    df = df[[col for col in spalten_zu_behalten if col in df.columns]]

# Update des Charts optional -> Monatsangaben
CHART_ID = "ekO2r"
dw.add_data(chart_id=CHART_ID, data=df)
dw.publish_chart(chart_id=CHART_ID, display=False)

In [ ]:
##Bestand
def generate_url():
    today = datetime.today()
    current_month = today.month
    year = today.year

    # Bestimme Quartal
    if current_month in [1, 2, 3]:
        quarter_start_month = 1
    elif current_month in [4, 5, 6]:
        quarter_start_month = 4
    elif current_month in [7, 8, 9]:
        quarter_start_month = 7
    else:
        quarter_start_month = 10

    url = f"https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ27/fz27_{year}{quarter_start_month:02d}.xlsx?__blob=publicationFile"
    response = requests.head(url)

    # Vorheriges Quartal, falls nicht verfügbar
    if response.status_code != 200:
        if quarter_start_month == 1:
            previous_year = year - 1
            url = f"https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ27/fz27_{previous_year}10.xlsx?__blob=publicationFile"
        else:
            previous_quarter_start_month = quarter_start_month - 3 if quarter_start_month > 1 else 10
            url = f"https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ27/fz27_{year}{previous_quarter_start_month:02d}.xlsx?__blob=publicationFile"

    return url

# -------------------------------
# 2. Excel-Datei herunterladen und Rohdaten einlesen
# -------------------------------
url_mob_best = generate_url()
response = requests.get(url_mob_best)
excel_data = BytesIO(response.content)

mob_best_raw = pd.read_excel(excel_data, sheet_name="FZ 27.9", engine='openpyxl', header=11).dropna(axis=1, how='all')

# Relevanten Bereich auswählen (bis zur ersten leeren Zeile)
empty_row_index = mob_best_raw.index[mob_best_raw.isnull().all(axis=1)].min()
mob_best_raw = mob_best_raw.iloc[:empty_row_index]

# Spalten umbenennen
neue_spalten_liste = ['Quartal', 'Anzahl insgesamt', 'AA - Anzahl insgesamt', 'AA - Anteil in %',
                     'EA - Anzahl insgesamt', 'EA - Anteil in %', 'batterieelektrischer Antrieb',
                     'Brennstoffzelle (Wasserstoff)', 'Plug-in-Hybrid', 'HoP - Anzahl insgesamt',
                     'Benzin-Hybrid', 'Diesel-Hybrid', 'Gas insgesamt', 'Wasserstoff']
mob_best_raw.rename(columns=dict(zip(mob_best_raw.columns, neue_spalten_liste)), inplace=True)

# -------------------------------
# 3. df_1 erstellen mit benötigten Spalten
# -------------------------------
df_1 = mob_best_raw[['Quartal', 'Benzin-Hybrid', 'Diesel-Hybrid']].copy()

# 'Quartal' → 'Jahr' umbenennen
df_1.rename(columns={'Quartal': 'Jahr'}, inplace=True)

# Nur Zeilen behalten, die auf 01.01.XXXX verweisen
df_1 = df_1[df_1['Jahr'].astype(str).str.contains(r'^01\.01\.(\d{4})$', na=False)].copy()

# Jahreszahl extrahieren
df_1['Jahr'] = df_1['Jahr'].astype(str).str.extract(r'(\d{4})')
df_1['Jahr'] = pd.to_numeric(df_1['Jahr'])

print(df_1)

## Zweiter Datensatz
def lade_excel(sheet="FZ 13.2.1"):
    jahr = datetime.now().year
    url_template = "https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ13/fz13_{jahr}.xlsx?__blob=publicationFile"

    while jahr >= 2000:
        r = requests.get(url_template.format(jahr=jahr))
        if r.status_code == 200:
            df = pd.read_excel(r.content, sheet_name=sheet, header=[7, 8], engine="openpyxl")
            df.columns = [
                ' '.join(str(c).strip().replace('\n', '') for c in col if pd.notna(c))
                if all("Unnamed:" not in str(c) for c in col) else str(col[0]).replace('\n', '')
                for col in df.columns
            ]
            df = df.iloc[:, 1:]  # erste Spalte entfernen
            return df, jahr

        jahr -= 1

    return None, None

# Nutzung
df, jahr = lade_excel()

if df is not None:
    df.columns = df.columns.str.strip()  # Entfernt führende und folgende Leerzeichen bei allen Spaltennamen
    df.rename(columns={"Jahr (jeweils 1. Juli/ab 20011. Januar)": "Jahr"}, inplace=True)

    if "Jahr" in df.columns:
        df = df[pd.to_numeric(df["Jahr"], errors="coerce") <= jahr] # Auswahl des richtigen Ausschnitts
        df = df[df['Jahr'] >= 2019].copy()

    # Operations
    # 1. Neue Spalte "Gas (LPG, CNG)" = Summe aus beiden Spalten
    df["Gas (LPG, CNG)"] = pd.to_numeric(df["Flüssiggas (LPG)(einschl. bivalent)"], errors='coerce').fillna(0) + pd.to_numeric(df["Erdgas (CNG)(einschl. bivalent)"], errors='coerce').fillna(0)


    # 2. New column "Hybrid o. Plug-in" = "Hybrid insgesamt" - "Hybrid dar. Plug-in"
    df["Hybrid (ohne Plug-in)"] = (
        pd.to_numeric(df["Hybrid insgesamt"], errors='coerce').fillna(0)
        - pd.to_numeric(df["Hybrid dar. Plug-in"], errors='coerce').fillna(0)
    )


    # 3. Spalten umbenennen
    df.rename(columns={"Hybrid dar. Plug-in": "Plug-in Hybrid "}, inplace=True)
    df.rename(columns={"Elektro(BEV)": "Elektro (BEV)"}, inplace=True)

    spalten_zu_behalten = ["Jahr","Benzin", "Diesel", "Elektro (BEV)", "Gas (LPG, CNG)", "Plug-in Hybrid ", "Sonstige", "Hybrid (ohne Plug-in)"]
    df = df[[col for col in spalten_zu_behalten if col in df.columns]]

    # Merge df with df_1 on the column 'Jahr'
    if df_1 is not None:
        # Convert 'Jahr' column in df_1 to numeric
        df_1['Jahr'] = pd.to_numeric(df_1['Jahr'], errors='coerce')
        # Merge df mit df_1 auf 'Jahr', um die Hybrid-Werte zu bekommen
        merged = df.merge(df_1[['Jahr', 'Diesel-Hybrid', 'Benzin-Hybrid']], on='Jahr', how='left')

        # Subtract the Hybrid values
        merged['Benzin'] = pd.to_numeric(merged['Benzin'], errors='coerce').fillna(0) + pd.to_numeric(merged['Benzin-Hybrid'], errors='coerce').fillna(0)
        merged['Diesel'] = pd.to_numeric(merged['Diesel'], errors='coerce').fillna(0) + pd.to_numeric(merged['Diesel-Hybrid'], errors='coerce').fillna(0)


        # Calculate 'Hybrid (ohne Plug-in)' in the merged DataFrame
        merged['Hybrid_sum'] = pd.to_numeric(merged['Diesel-Hybrid'], errors='coerce').fillna(0) + pd.to_numeric(merged['Benzin-Hybrid'], errors='coerce').fillna(0)
        merged['Hybrid (ohne Plug-in)'] = pd.to_numeric(merged['Hybrid (ohne Plug-in)'], errors='coerce').fillna(0) - merged['Hybrid_sum']


        # Differenz zu bestehender 'Sonstige'-Spalte addieren
        merged['Sonstige'] = pd.to_numeric(merged['Sonstige'], errors='coerce').fillna(0) + merged['Hybrid (ohne Plug-in)']


        # Optional: keine negativen Werte erlauben
        merged['Sonstige'] = merged['Sonstige'].apply(lambda x: x if x > 0 else 0)


        # Hilfsspalten entfernen
        merged.drop(columns=['Diesel-Hybrid', 'Benzin-Hybrid', 'Hybrid_sum', 'Hybrid (ohne Plug-in)'], inplace=True)


        # Ergebnis zurück in df speichern
        df = merged
    spalten_zu_behalten = ["Jahr", "Benzin", "Diesel", "Elektro (BEV)", "Gas (LPG, CNG)", "Plug-in Hybrid ", "Sonstige"]
    df = df[[col for col in spalten_zu_behalten if col in df.columns]]

# Update des Charts optional -> Monatsangaben
CHART_ID = "kVNCB"
dw.add_data(chart_id=CHART_ID, data=df)
dw.publish_chart(chart_id=CHART_ID, display=False)

In [ ]:
def lade_excel(sheet="FZ 13.2.1"):
    jahr = datetime.now().year
    url_template = "https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ13/fz13_{jahr}.xlsx?__blob=publicationFile"

    while jahr >= 2000:
        r = requests.get(url_template.format(jahr=jahr))
        if r.status_code == 200:
            df = pd.read_excel(r.content, sheet_name=sheet, header=[7, 8], engine="openpyxl")
            df.columns = [
                ' '.join(str(c).strip().replace('\n', '') for c in col if pd.notna(c))
                if all("Unnamed:" not in str(c) for c in col) else str(col[0]).replace('\n', '')
                for col in df.columns
            ]
            df = df.iloc[:, 1:]  # erste Spalte entfernen
            return df, jahr

        jahr -= 1

    return None, None

# Lade Deckblatt um Datum zu bekommen
def lade_excel_stand(sheet="Deckblatt"):
    jahr = datetime.now().year
    url_template = "https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ13/fz13_{jahr}.xlsx?__blob=publicationFile"

    while jahr >= 2000:
        r = requests.get(url_template.format(jahr=jahr))
        if r.status_code == 200:
            df = pd.read_excel(r.content, sheet_name=sheet, header=[16], engine="openpyxl")

            return df, jahr

        jahr -= 1

    return None, None
# Nutzung
df, jahr = lade_excel()
df_stand, jahr = lade_excel_stand()
date_col = df_stand.columns

df.columns = df.columns.str.strip()  # Entfernt führende und folgende Leerzeichen bei allen Spaltennamen
df.rename(columns={"Jahr (jeweils 1. Juli/ab 20011. Januar)": "Jahr"}, inplace=True)

if "Jahr" in df.columns:
    df = df[pd.to_numeric(df["Jahr"], errors="coerce") <= jahr] # Auswahl des richtigen Ausschnitts

# Operationen
# 1. Neue Spalte "Gas (LPG, CNG)" = Summe aus beiden Spalten
df["Gas (LPG, CNG)"] = df["Flüssiggas (LPG)(einschl. bivalent)"].fillna(0) + df["Erdgas (CNG)(einschl. bivalent)"].fillna(0)

# 2. Neue Spalte "Hybrid o. Plug-in" = "Hybrid insgesamt" - "Hybrid dar. Plug-in"
df["Hybrid (ohne Plug-in)"] = (
    pd.to_numeric(df["Hybrid insgesamt"], errors='coerce').fillna(0)
    - pd.to_numeric(df["Hybrid dar. Plug-in"], errors='coerce').fillna(0)
)

# 3. Spalten umbenennen
df.rename(columns={"Hybrid dar. Plug-in": "Plug-in Hybrid "}, inplace=True)
df.rename(columns={"Elektro(BEV)": "Elektro (BEV)"}, inplace=True)

spalten_zu_behalten = ["Jahr", "Elektro (BEV)"]
df = df[[col for col in spalten_zu_behalten if col in df.columns]]

langfrist_o45_values = {
    2025: 3379131,
    2030: 11713044,
    2035: 24113939,
    2040: 34046623,
    2045: 39603178
}

mms_values = {
    2025: 2691345,
    2030: 8705133,
    2035: 18908938,
    2040: 28519883,
    2045: 35272876
}

# Add langfrist values to df

#Create full range of years from earliest year to 2045
full_years = pd.DataFrame({"Jahr": range(int(df["Jahr"].min()), 2046)})
df = full_years.merge(df, on="Jahr", how="left")

# 4. Add langfrist_o45_values and mms values
langfrist_df = pd.DataFrame(list(langfrist_o45_values.items()), columns=["Jahr", "Langfrist O45-Strom"])
df = df.merge(langfrist_df, on="Jahr", how="left")

mms_df = pd.DataFrame(list(mms_values.items()), columns=["Jahr", "MMS"])
df = df.merge(mms_df, on="Jahr", how="left")

tag, monat, jahr = date_col[0].split(sep=' ')

monat_as_digit = german_month_to_digit(monat)

notes_text = f'Stand: {str(monat_as_digit.strip())}.{jahr}'

# Update des Charts optional -> Monatsangaben
CHART_ID = "Oj4EI"
dw.add_data(chart_id=CHART_ID, data=df)
dw.update_chart(CHART_ID, metadata={
    'annotate': {
        'notes': notes_text
    }
})
#dw.update_description(chart_id=CHART_ID, source_name=f"KBA (Stand {col[0]})")
dw.publish_chart(chart_id=CHART_ID, display=False)

In [ ]:
# Check if excel file has already been read in in previous cell
try:
  url_bna_lsi
  lsi_data_full = pd.read_excel(data_bna_lsi, sheet_name='3.1 Ladepunkte') # Load all of sheet 4

# If not, fetch from url
except NameError:
  url_bna_lsi = "https://www.bundesnetzagentur.de/DE/Fachthemen/ElektrizitaetundGas/E-Mobilitaet/_DL/Ladesaeuleninfrastruktur.xlsx?__blob=publicationFile&"

  response = requests.get(url_bna_lsi)
  excel_data_bna_lsi = BytesIO(response.content)

  # Einlesen der Daten aus Excel-Datei
  data_bna_lsi = pd.ExcelFile(excel_data_bna_lsi)
  lsi_data_full = pd.read_excel(data_bna_lsi , sheet_name='3.1 Ladepunkte', )

# Transpose and drop first 4 columns
lsi_data = lsi_data_full.transpose()
lsi_data = lsi_data.iloc[4:]

# Keep relevant columns only
columns_to_keep = [5, 8]
lsi_data_y = lsi_data[columns_to_keep]

# Rename columns
lsi_data_y.columns = ['Datum', 'Anzahl Ladepunkte']
lsi_data_y['Datum'] = pd.to_datetime(lsi_data_y['Datum'][1:])

# Drop first row, reset index
lsi_data_y.index = range(len(lsi_data_y))
lsi_data_y = lsi_data_y.drop([0])
lsi_data_y.index = range(len(lsi_data_y))

date_range = pd.date_range(start='2021-01-01', end='2030-01-01', freq='3MS')
last_year = lsi_data_y['Datum'].max()

# Create a DataFrame with these dates
new_data = pd.DataFrame({'Datum': date_range})

# Define the starting and ending values for KV Ziele
#kv_start = 45000
#kv_end = 1000000

# Create the KV Ziele column with linear interpolation
# Calculate the total number of periods (months) between the start and end date
#total_periods = len(date_range)

# Linearly interpolate between kv_start and kv_end over the total periods
#new_data['Ziele (laut Koalitionsvertrag 21)'] = np.linspace(kv_start, kv_end, total_periods)

# Reset index to clean up the DataFrame
#new_data.reset_index(drop=True, inplace=True)

#lsi_data_y = pd.merge(lsi_data_y, new_data, on='Datum', how='outer', sort=False)

# Langfrist-Szenario (Zielwerte zu bestimmten Stichtagen)
langfristszenario_o45_ldp = {
    '2025-12-01': 172581.075,
    '2030-12-01': 497466.99,
    '2035-12-01': 941144.85,
    '2040-12-01': 1237114.616,
    '2045-12-01': 1366530.853
}

# In DataFrame umwandeln
langfrist_df = pd.DataFrame({
    'Datum': pd.to_datetime(list(langfristszenario_o45_ldp.keys())),
    'Langfristszenario O45-Strom': list(langfristszenario_o45_ldp.values())
})

# Merge mit Hauptdatensatz
lsi_data_y = pd.merge(lsi_data_y, langfrist_df, on='Datum', how='outer')

# Sortieren und Index neu setzen
lsi_data_y = lsi_data_y.sort_values(by='Datum').reset_index(drop=True)

# Remove langfristziele for previous years
#lsi_data_y.loc[lsi_data_y['Datum'] < '2025-01-01', 'Langfristszenario O45-Strom'] = np.nan

# Generate headerf
#condition = lsi_data_y['Datum'] == last_year
#last_year_lsi = lsi_data_y.loc[condition, 'Anzahl Ladepunkte'].values[0]

#condition = lsi_data_y['Datum'] == last_year - pd.DateOffset(years=1)
#previous_year_lsi = lsi_data_y.loc[condition, 'Anzahl Ladepunkte'].values[0]

#condition2 = lsi_data_y['Datum'] == last_year - pd.DateOffset(years=2)
#previous_previous_year_lsi = lsi_data_y.loc[condition2, 'Anzahl Ladepunkte'].values[0]

#condition3 = lsi_data_y['Datum'] == str(last_year - pd.DateOffset(years=1))[:4]
#previous_year_lsi_end = lsi_data_y.loc[condition3, 'Anzahl Ladepunkte'].values[0]

#condition4 = lsi_data_y['Datum'] == str(last_year - pd.DateOffset(years=2))[:4]
#previous_year_lsi_start = lsi_data_y.loc[condition4, 'Anzahl Ladepunkte'].values[0]

#lsi_unterschied = round((int(previous_year_lsi_end) - int(previous_year_lsi_start)), 1)
#formatted_lsi_unterschied = locale.format_string("%d", lsi_unterschied, grouping=True)
#unterschied_wort = 'hinzugebaut'
#if lsi_unterschied < 0:
#  unterschied_wort = 'außer Betrieb genommen'

#last_year_date = datetime_to_german_date(str(last_year))
#previous_year_date = datetime_to_german_date(str(last_year - pd.DateOffset(years=1)))

#lsi_header = f' Die Anzahl öffentlicher Ladepunkte lag im {last_year_date} bei {format_german_number(last_year_lsi)}. In {previous_year_date[-4:]} wurden netto insgesamt {format_german_number(lsi_unterschied)} Ladepunkte {unterschied_wort}. Das Ziel aus dem Koalitionsvertrag 21 für 2030 lag bei 1 Mio. öffentlichen Ladepunkten.'

# Zeile mit dem letzten verfügbaren Wert in 'Anzahl Ladepunkte' finden
letzter_wert = lsi_data_y[lsi_data_y['Anzahl Ladepunkte'].notna()].iloc[-1]
zeitstempel = letzter_wert['Datum']
zeitstempel_str = f"Stand: {zeitstempel.strftime('%m.%Y')}"
notes_text = f"{zeitstempel_str}"


CHART_ID = 'clw08'
dw.add_data(chart_id=CHART_ID, data=lsi_data_y)
dw.update_chart(CHART_ID, metadata={
    'annotate': {
        'notes': notes_text
    }
})
dw.publish_chart(chart_id=CHART_ID, display=False)